In [ ]:
# pip install pandas

In [1]:
import pandas as pd
import numpy as np
from glob import glob
import os, shutil
from os.path import join, basename, splitext, dirname
from xml.etree.ElementTree import ElementTree

In [2]:
def mkdirs(path_dir):
    if not os.path.exists(path_dir):
        os.makedirs(path_dir)
        
def get_data(image):
    """이미지 파일 경로를 받아 xml을 읽고 필요한 정보를 가져옵니다."""
    breed_folder = basename(dirname(image))
    xml_file = join(Annotations_folder, breed_folder, f'{basename(splitext(image)[0])}.xml')
    
    root = ElementTree().parse(xml_file)

    object_ = root.find('object')
    name = object_.find('name').text
    bndbox = object_.find('bndbox')
    xmin, ymin, xmax, ymax = map(lambda e: int(e.text), bndbox[:4])
    
    return join(breed_folder, basename(image)), xmin, ymin, xmax, ymax, name

In [3]:
# 품종 - 폴더 이름
path_to_dataset = r'C:\project\Machine Learning Project\yolov3\tf2-keras-yolo3-master\견종_10종'
Final_folder = join(path_to_dataset, '최종')
Crop_folder = join(path_to_dataset, '수작업 처리 후')
Annotations_folder = join(path_to_dataset, 'Annotations')

In [4]:
X = glob(join(Crop_folder, '*', '*.jpg'))
y = np.array([get_data(image) for image in X])

In [5]:
# DataFrame 객체 생성
df = pd.DataFrame(data=y, columns=['image', 'xmin', 'ymin', 'xmax', 'ymax', 'breed'])
df

,image,xmin,ymin,xmax,ymax,breed
0,#beaglelove\2022-02-03 10.30.07 27651518371566...,156,90,858,444,BEAGLE
1,#beaglelove\2022-02-14 01.28.57 27728519918797...,10,137,1190,1642,BEAGLE
2,#beaglelove\2022-02-14 05.48.11 27729824607300...,521,476,1051,1339,BEAGLE
3,#beaglelove\2022-02-14 05.48.11 27729824607467...,550,792,1273,1366,BEAGLE
4,#beaglelove\2022-02-21 00.05.22 27778833553337...,305,498,430,748,BEAGLE
...,...,...,...,...,...,...
5892,#welshcorgi\2022-03-06 12.21.22 27876758715999...,94,365,299,627,WELSH CORGI
5893,#welshcorgi\2022-03-06 12.35.57 27876832149052...,5,212,1362,1799,WELSH CORGI
5894,#welshcorgi\2022-03-06 12.35.57 27876832149136...,11,101,1343,1777,WELSH CORGI
5895,#welshcorgi\2022-03-06 12.35.57 27876832150310...,1,145,1326,1778,WELSH CORGI


In [6]:
# 데이터 타입 변환
for column in ['xmin', 'ymin', 'xmax', 'ymax']:
    df[column] = df[column].astype('int32')
    
df.dtypes

image    object
xmin      int32
ymin      int32
xmax      int32
ymax      int32
breed    object
dtype: object

In [7]:
# csv로 저장
df.to_csv(join(Final_folder, 'dog.csv'))

In [9]:
# 원본 이미지 복사
for image in df.image:
    mkdirs(join(Final_folder, basename(dirname(image))))
    original = join(path_to_dataset, '원본', image)
    copy = join(Final_folder, image)
    shutil.copy(original, copy)
    # print(original, copy)